In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np


from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics.pairwise import cosine_distances as cd
from collections import defaultdict
from pyeer.eer_info import get_eer_stats
from sklearn.metrics.pairwise import manhattan_distances as md

from sklearn.metrics.pairwise import cosine_similarity as cs

from pytorch_metric_learning.distances import LpDistance, CosineSimilarity,SNRDistance

def EERf(results):
    results = np.array(results)
    genuine = results[results[:, 1] == 1][:, 0]
    impostor = results[results[:, 1] == 0][:, 0]
    stats_a = get_eer_stats(genuine, impostor)
    return stats_a.eer, stats_a.fmr100, stats_a.fmr1000, stats_a.fmr10000

def calculate_and_print_averages(y_train, results3):
    u, counts = np.unique(y_train, return_counts=True)
    eer_values = []
    fmr100_values = []
    fmr1000_values = []
    fmr10000_values = []
    ii = 0

    for i in results3.keys():
        re = EERf(results3[i])
        eer = re[0]
        fmr100 = re[1]
        fmr1000 = re[2]
        fmr10000 = re[3]
        
        print(f"{i}: EER = {re[0]:.4f}, FMR100 = {re[1]:.4f}, FMR1000 = {re[2]:.4f}, FMR10000 = {re[3]:.4f}, Count = {counts[ii]}")
        
        eer_values.append(eer)
        fmr100_values.append(fmr100)
        fmr1000_values.append(fmr1000)
        fmr10000_values.append(fmr10000)
        ii += 1

    # Calculate averages and standard deviations
    average_eer = np.mean(eer_values) * 100
    std_eer = np.std(eer_values) * 100
    average_fmr100 = np.mean(fmr100_values) * 100
    std_fmr100 = np.std(fmr100_values) * 100
    average_fmr1000 = np.mean(fmr1000_values) * 100
    std_fmr1000 = np.std(fmr1000_values) * 100
    average_fmr10000 = np.mean(fmr10000_values) * 100
    std_fmr10000 = np.std(fmr10000_values) * 100

    # Print results
    print(f"Final Average EER: {average_eer:.4f}")
    print(f"Final EER Standard Deviation: {std_eer:.4f}")
    print(f"${average_eer:.2f} \\pm {std_eer:.2f}$")
    print(f"Final Average FMR100: {average_fmr100:.4f}")
    print(f"Final FMR100 Standard Deviation: {std_fmr100:.4f}")
    print(f"${average_fmr100:.2f} \\pm {std_fmr100:.2f}$")
    print(f"Final Average FMR1000: {average_fmr1000:.4f}")
    print(f"Final FMR1000 Standard Deviation: {std_fmr1000:.4f}")
    print(f"${average_fmr1000:.2f} \\pm {std_fmr1000:.2f}$")
    print(f"Final Average FMR10000: {average_fmr10000:.4f}")
    print(f"Final FMR10000 Standard Deviation: {std_fmr10000:.4f}")
    print(f"${average_fmr10000:.2f} \\pm {std_fmr10000:.2f}$")
    
    return average_eer, std_eer, average_fmr100, std_fmr100, average_fmr1000, std_fmr1000, average_fmr10000, std_fmr10000


import torch.nn.functional as F





def calculate_similarity_scores_two(enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance):
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]

    if distance == "cd":
        similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    elif distance == "ed":
        similarity_matrix = -1 * ed(verification_embeddings, enrollment_embeddings)

    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        predicted_scores = similarity_matrix[i]
        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]

        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            max_score = sum(sorted(predicted_scores[same_class_indices], reverse=True)[:10]) / 10
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls, i, cls])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls, i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two(enrollment_data, ye, verification_data, yv, e_network, distance):
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two(
        enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance
    )
    return similarity_results_by_class, similarity_results_by_class_dict




import torch
import numpy as np

def compute_embedding_batch_two(x_test_batch, embedding_network, batch_size=150, device="cuda"):
    """
    Computes embeddings for the input data in batches, including batch-wise tensor conversion.

    Args:
        x_test_batch (numpy.ndarray): Input data (in numpy format).
        embedding_network (torch.nn.Module): The network to compute embeddings.
        batch_size (int, optional): Size of each batch for processing. Default is 100.
        device (str, optional): The device to use ('cuda' or 'cpu'). Default is 'cuda'.

    Returns:
        numpy.ndarray: Computed embeddings for the input data.
    """
    total_samples = len(x_test_batch)
    embeddings = []

    # Process the data in batches
    for start_index in range(0, total_samples, batch_size):
        end_index = min(start_index + batch_size, total_samples)

        # Convert the current batch to tensor and move to device
        batch = torch.tensor(x_test_batch[start_index:end_index], dtype=torch.float32).to(device)

        with torch.no_grad():  # Disable gradient computation for inference
            batch_embeddings = embedding_network(batch).detach().cpu().numpy()

        embeddings.append(batch_embeddings)  # Store embeddings on CPU

        # Clear GPU memory for the batch
        del batch
        torch.cuda.empty_cache()

    # Concatenate all embeddings into a single numpy array
    anchor_embeddings = np.concatenate(embeddings, axis=0)

    print(f"Computed embeddings shape: {anchor_embeddings.shape}")
    return anchor_embeddings

In [2]:
import torch
import torch.nn as nn

class EEGNet7(nn.Module):
    def __init__(self):
        super(EEGNet7, self).__init__()
        # InstanceNorm1d normalizes each channel across its 500 values for each sample
        self.norm = nn.InstanceNorm1d(93, affine=False)  # Normalizes across the 500 values in each of the 93 channels
        self.act = nn.ReLU()  # or nn.SELU() , ReLU
        self.act2 = nn.Tanh()
        self.conv1 = nn.Conv2d(1, 256, (1, 4), padding='same')
        self.pool1 = nn.MaxPool2d((1, 2))
        self.conv2 = nn.Conv2d(256, 192, (4, 1), padding='same')
        self.pool2 = nn.MaxPool2d((2, 1))
        self.conv3 = nn.Conv2d(192, 128, (1, 4), padding='same')
        self.pool3 = nn.MaxPool2d((1, 2))
        self.conv4 = nn.Conv2d(128, 96, (4, 1), padding='same')
        self.pool4 = nn.MaxPool2d((2, 1))
        self.conv5 = nn.Conv2d(96, 64, (1, 4), padding='same')
        self.pool5 = nn.MaxPool2d((1, 2))
        self.conv6 = nn.Conv2d(64, 32, (4, 1), padding='same')
        self.pool6 = nn.MaxPool2d((2, 1))
        self.conv7 = nn.Conv2d(32, 16, (1, 2), padding='same')
        self.conv8 = nn.Conv2d(16, 2, (2, 1), padding='same')
        self.flatten = nn.Flatten()
        self.embedding = nn.Linear(1364, 128)  # Embedding layer

    def forward(self, x):
        #if x.dim() == 3:
            #x = x.unsqueeze(1)  # Adding a channel dimension if input is 3D
        x = self.norm(x)  # Apply InstanceNorm along the channel dimension (squeeze to 1D first)
        x = x.unsqueeze(1)  # Reshape to 4D again for Conv2d
        x = self.act(self.conv1(x))
        x = self.pool1(x)
        x = self.act(self.conv2(x))
        x = self.pool2(x)
        x = self.act(self.conv3(x))
        x = self.pool3(x)
        x = self.act(self.conv4(x))
        x = self.pool4(x)
        x = self.act(self.conv5(x))
        x = self.pool5(x)
        x = self.act(self.conv6(x))
        x = self.pool6(x)
        x = self.act(self.conv7(x))
        x = self.act(self.conv8(x))
        x = self.flatten(x)
        x = self.embedding(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trunk = EEGNet7().to(device)

In [ ]:
# import h5py
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, Subset
import h5py
import numpy as np

import h5py

import h5py
import numpy as np

# Load test data from test_raw.h5
with h5py.File("./data/test_raw.h5", "r") as f_test:
    X_test = f_test['data'][:]
    Y_test = f_test['labels'][:]
    S_test = f_test['sessions'][:]
    H_test = f_test['hardwares'][:]  # Load the 'hardwares' dataset as h_test

# Load negative data from neg_raw.h5
with h5py.File("./data/neg_raw.h5", "r") as f_neg:
    X_neg = f_neg['data'][:]
    Y_neg = f_neg['labels'][:]
    S_neg = f_neg['sessions'][:]
    H_neg = f_neg['hardwares'][:]  # Load the 'hardwares' dataset as h_neg

# Optional: Combine or use them independently depending on your requirement
# Example of concatenating them for a combined dataset
X_test = np.concatenate((X_test, X_neg), axis=0)
Y_test = np.concatenate((Y_test, Y_neg), axis=0)
S_test = np.concatenate((S_test, S_neg), axis=0)
H_test = np.concatenate((H_test, H_neg), axis=0)

# Now, X_combined, Y_combined, S_combined, and h_combined contain the merged data


# Now, X_combined, Y_combined, and S_combined contain the merged data


# Shuffle the data
indices = np.arange(X_test.shape[0])
np.random.shuffle(indices)

# Reorder the arrays according to the shuffled indices
x_test = X_test[indices]
y_test = Y_test[indices]
s_test = S_test[indices]
h_test = H_test[indices]


# Find unique subjects
unique_subjects = np.unique(y_test)

# Initialize lists to hold the data for x_test_e and x_test_v
x_test_e_list = []
y_test_e_list = []
s_test_e_list = []
h_test_e_list = []

x_test_v_list = []
y_test_v_list = []
s_test_v_list = []
h_test_v_list = []

# Assign the minimum session for each subject to x_test_e and the rest to x_test_v
for subject in unique_subjects:
    subject_indices = np.where(y_test == subject)[0]
    subject_sessions = s_test[subject_indices]
    
    # Skip subjects with fewer than two unique sessions
    if len(np.unique(subject_sessions)) < 2:
        continue

    print(f"Subject {subject}, Number of unique sessions: {len(np.unique(subject_sessions))}")
    
    # Assign the minimum session to the evaluation set (x_test_e)
    min_session = np.min(subject_sessions)
    
    # Append data to the evaluation set (min session)
    x_test_e_list.extend(x_test[subject_indices][subject_sessions == min_session])
    y_test_e_list.extend(y_test[subject_indices][subject_sessions == min_session])
    s_test_e_list.extend(s_test[subject_indices][subject_sessions == min_session])
    h_test_e_list.extend(h_test[subject_indices][subject_sessions == min_session])

    # Append remaining sessions to the validation set (x_test_v)
    x_test_v_list.extend(x_test[subject_indices][subject_sessions != min_session])
    y_test_v_list.extend(y_test[subject_indices][subject_sessions != min_session])
    s_test_v_list.extend(s_test[subject_indices][subject_sessions != min_session])
    h_test_v_list.extend(h_test[subject_indices][subject_sessions != min_session])

# Shuffle and convert lists back to numpy arrays for x_test_e
indices_e = np.arange(len(x_test_e_list))
np.random.shuffle(indices_e)

x_test_e = np.array(x_test_e_list)[indices_e]
y_test_e = np.array(y_test_e_list)[indices_e]
s_test_e = np.array(s_test_e_list)[indices_e]
h_test_e = np.array(h_test_e_list)[indices_e]

# Shuffle and convert lists back to numpy arrays for x_test_v
indices_v = np.arange(len(x_test_v_list))
np.random.shuffle(indices_v)

x_test_v = np.array(x_test_v_list)[indices_v]
y_test_v = np.array(y_test_v_list)[indices_v]
s_test_v = np.array(s_test_v_list)[indices_v]
h_test_v = np.array(h_test_v_list)[indices_v]
# Optional: Save the new test evaluation and validation sets to npy files (if needed)








print(f"x_test_e: {x_test_e.shape}, y_test_e: {y_test_e.shape}, s_test_e: {s_test_e.shape}")
print(f"x_test_v: {x_test_v.shape}, y_test_v: {y_test_v.shape}, s_test_v: {s_test_v.shape}")


# Load the state dictionary from the saved file
state_dict = torch.load('./model/SupConLoss128_m4_e99_unconnected.pth')

# Load the state dictionary into the model
trunk.load_state_dict(state_dict)



x_test_e = compute_embedding_batch_two(x_test_e, trunk)
x_test_v = compute_embedding_batch_two(x_test_v, trunk)

#np.save('x_test_e.npy', x_test_ee)
#np.save('y_test_e.npy', y_test_e)
#np.save('s_test_e.npy', s_test_e)
#np.save('h_test_e.npy', h_test_e)


#np.save('x_test_v.npy', x_test_ve)
#np.save('y_test_v.npy', y_test_v)
#np.save('s_test_v.npy', s_test_v)
#np.save('h_test_v.npy', h_test_v)

Subject 70, Number of unique sessions: 19
Subject 82, Number of unique sessions: 19
Subject 85, Number of unique sessions: 19
Subject 86, Number of unique sessions: 19
Subject 96, Number of unique sessions: 18
Subject 103, Number of unique sessions: 18
Subject 106, Number of unique sessions: 41
Subject 109, Number of unique sessions: 5
Subject 111, Number of unique sessions: 19
Subject 115, Number of unique sessions: 41
Subject 118, Number of unique sessions: 19
Subject 119, Number of unique sessions: 8
Subject 125, Number of unique sessions: 19
Subject 129, Number of unique sessions: 5
Subject 130, Number of unique sessions: 19
Subject 131, Number of unique sessions: 19
Subject 136, Number of unique sessions: 19
Subject 145, Number of unique sessions: 19
Subject 156, Number of unique sessions: 6
Subject 159, Number of unique sessions: 19
Subject 167, Number of unique sessions: 2
Subject 169, Number of unique sessions: 6
Subject 173, Number of unique sessions: 6
Subject 174, Number of 

In [3]:
import numpy as np

# Loading the files with the same names
x_test_e = np.load('x_test_e.npy')
y_test_e = np.load('y_test_e.npy')
s_test_e = np.load('s_test_e.npy')
h_test_e = np.load('h_test_e.npy')

x_test_v = np.load('x_test_v.npy')
y_test_v = np.load('y_test_v.npy')
s_test_v = np.load('s_test_v.npy')
h_test_v = np.load('h_test_v.npy')

# Verify loaded arrays (optional)
print(x_test_e.shape, y_test_e.shape, s_test_e.shape, h_test_e.shape)
print(x_test_v.shape, y_test_v.shape, s_test_v.shape, h_test_v.shape)



def compute_embedding_batch_two(x_test_batch, embedding_network, batch_size=100, device="cuda"):
    #print(x_test_batch.shape)
    return x_test_batch


(10000, 128) (10000,) (10000,) (10000,)
(160051, 128) (160051,) (160051,) (160051,)


In [ ]:
def compute_embedding_batch_two(x_test_batch, embedding_network, batch_size=100, device="cuda"):
    #print(x_test_batch.shape)
    return x_test_batch


In [ ]:
x_test_v.shape

In [ ]:

#, s_test_b
resutls2,resutls3=assessment_model_data_two(x_test_e,y_test_e, x_test_v,y_test_v,trunk, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

In [14]:
def calculate_similarity_scores_two2(enrollment_embeddings,y_enrollment, verification_embeddings, y_verification,s_ver, extra= 3):
    flag_s=0
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    print(unique_classes)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    # Compute similarity matrix
    print("I am 9")
    #similarity_matrix =batch_pairwise_distances(verification_embeddings, enrollment_embeddings, 100)

    print(verification_embeddings.shape, enrollment_embeddings.shape )
    
    similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    #similarity_matrix = abs(cs(verification_embeddings, enrollment_embeddings))
    print("I am 10")
    print(similarity_matrix.shape)
    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        subject_session = s_ver[i]
        #current_ss= np.where((s_ver == subject_session) & (y_verification == current_class))
        
        current_ss = np.where((s_ver == subject_session) & (y_verification == current_class))[0]  # assuming s_ver == subject_session is an array of booleans
        #print("aa", len(current_ss))
        
        selected_indices = np.random.choice(current_ss, extra, replace=True)
        selected_indices = np.append(selected_indices, i)

        #print(current_class, y_verification[selected_indices], s_ver[selected_indices], selected_indices)

        predicted_scoresg = similarity_matrix[selected_indices]
        


        
        predicted_scores = similarity_matrix[i]


        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]


        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #max_score = np.mean(predicted_scores[same_class_indices])
            #max_score = max(predicted_scores[same_class_indices])
            maxscore=[]
            for ir in range(extra+1):
                max_score = sum(sorted(predicted_scoresg[ir, same_class_indices], reverse=True)[:10]) / 10
                maxscore.append(max_score)
            max_score = sum(maxscore)/(extra+1)
            
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls,i, cls ])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls,i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two2(enrollment_data, ye, verification_data,yv, e_network, s_ver, distance = "cd"):
    x_enrollment, y_enrollment = enrollment_data , ye
    x_verification, y_verification = verification_data, yv


    print(f"Total number of enrollment samples: {len(x_enrollment)}")
    print(f"Total number of verification samples: {len(x_verification)}", verification_data.shape)

    # Compute embeddings for enrollment data
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    print("I am 0 - enrollment", enrollment_embeddings.shape)
    # Compute embeddings for verification data
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    print("I am 1")

    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two2(
        enrollment_embeddings,y_enrollment, verification_embeddings, y_verification, s_ver
    )

    return similarity_results_by_class, similarity_results_by_class_dict

In [15]:

#, s_test_b
resutls2,resutls3=assessment_model_data_two2(x_test_e,y_test_e, x_test_v,y_test_v,trunk, s_test_v, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

Total number of enrollment samples: 10000
Total number of verification samples: 160051 (160051, 128)
I am 0 - enrollment (10000, 128)
I am 1
[ 70  82  85  86  96 103 106 109 111 115 118 119 125 129 130 131 136 145
 156 159 167 169 173 174 175 176 181 183 184 187 191 192 194 197 199 201
 203 206 207 214 216 217 219 221 225 230 231 233 236 245 249 250 251 257
 260 261 262 265 268 270 277 281 282 284 286 291 296 299 301 303 306 310
 314 322 325 332 336 342 348 354 357 364 365 373 374 384 385 387 402 404
 407 413 414 415 418 419 422 423 433 436]
I am 9
(160051, 128) (10000, 128)
I am 10
(160051, 10000)
70: EER = 0.1883, FMR100 = 0.9122, FMR1000 = 0.9794, FMR10000 = 0.9961, Count = 100
82: EER = 0.1294, FMR100 = 0.8542, FMR1000 = 0.9819, FMR10000 = 0.9953, Count = 100
85: EER = 0.2202, FMR100 = 0.9506, FMR1000 = 0.9983, FMR10000 = 1.0000, Count = 100
86: EER = 0.0956, FMR100 = 0.6678, FMR1000 = 0.9661, FMR10000 = 0.9928, Count = 100
96: EER = 0.2249, FMR100 = 0.7829, FMR1000 = 0.9388, FMR10

/home/fallahi/anaconda3/lib/python3.12/site-packages/pyeer/eer_stats.py:220: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


306: EER = 0.6377, FMR100 = 0.9477, FMR1000 = 0.9650, FMR10000 = 0.9950, Count = 100
310: EER = 0.2486, FMR100 = 0.9582, FMR1000 = 0.9986, FMR10000 = 1.0000, Count = 100
314: EER = 0.1809, FMR100 = 0.7660, FMR1000 = 0.8840, FMR10000 = 0.9720, Count = 100
322: EER = 0.2956, FMR100 = 0.9770, FMR1000 = 0.9955, FMR10000 = 0.9990, Count = 100
325: EER = 0.2759, FMR100 = 0.9557, FMR1000 = 0.9986, FMR10000 = 1.0000, Count = 100
332: EER = 0.2839, FMR100 = 0.7233, FMR1000 = 0.9533, FMR10000 = 1.0000, Count = 100
336: EER = 0.1810, FMR100 = 0.9191, FMR1000 = 0.9895, FMR10000 = 1.0000, Count = 100
342: EER = 0.1369, FMR100 = 0.9645, FMR1000 = 0.9973, FMR10000 = 0.9995, Count = 100
348: EER = 0.1837, FMR100 = 0.9827, FMR1000 = 0.9991, FMR10000 = 1.0000, Count = 100
354: EER = 0.1137, FMR100 = 0.7850, FMR1000 = 0.9686, FMR10000 = 0.9941, Count = 100
357: EER = 0.1819, FMR100 = 0.9681, FMR1000 = 0.9952, FMR10000 = 0.9990, Count = 100
364: EER = 0.1176, FMR100 = 0.8781, FMR1000 = 0.9862, FMR10000 = 

(23.84431819202313,
 10.521660743194138,
 89.54468162249903,
 12.61471388602599,
 98.01318649216408,
 3.5232474293513234,
 99.6148095441336,
 0.9476061909337592)

In [7]:
def calculate_similarity_scores_two2(enrollment_embeddings,y_enrollment, verification_embeddings, y_verification,s_ver, extra= 4):
    flag_s=0
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    print(unique_classes)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    # Compute similarity matrix
    print("I am 9")
    #similarity_matrix =batch_pairwise_distances(verification_embeddings, enrollment_embeddings, 100)

    print(verification_embeddings.shape, enrollment_embeddings.shape )
    
    similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    #similarity_matrix = abs(cs(verification_embeddings, enrollment_embeddings))
    print("I am 10")
    print(similarity_matrix.shape)
    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        subject_session = s_ver[i]
        #current_ss= np.where((s_ver == subject_session) & (y_verification == current_class))
        
        current_ss = np.where((s_ver == subject_session) & (y_verification == current_class))[0]  # assuming s_ver == subject_session is an array of booleans
        #print("aa", len(current_ss))
        
        selected_indices = np.random.choice(current_ss, extra, replace=True)
        selected_indices = np.append(selected_indices, i)

        #print(current_class, y_verification[selected_indices], s_ver[selected_indices], selected_indices)

        predicted_scoresg = similarity_matrix[selected_indices]
        


        
        predicted_scores = similarity_matrix[i]


        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]


        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #max_score = np.mean(predicted_scores[same_class_indices])
            #max_score = max(predicted_scores[same_class_indices])
            maxscore=[]
            for ir in range(extra+1):
                max_score = sum(sorted(predicted_scoresg[ir, same_class_indices], reverse=True)[:10]) / 10
                maxscore.append(max_score)
            max_score = sum(maxscore)/(extra+1)
            
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls,i, cls ])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls,i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two2(enrollment_data, ye, verification_data,yv, e_network, s_ver, distance = "cd"):
    x_enrollment, y_enrollment = enrollment_data , ye
    x_verification, y_verification = verification_data, yv


    print(f"Total number of enrollment samples: {len(x_enrollment)}")
    print(f"Total number of verification samples: {len(x_verification)}", verification_data.shape)

    # Compute embeddings for enrollment data
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    print("I am 0 - enrollment", enrollment_embeddings.shape)
    # Compute embeddings for verification data
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    print("I am 1")

    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two2(
        enrollment_embeddings,y_enrollment, verification_embeddings, y_verification, s_ver
    )

    return similarity_results_by_class, similarity_results_by_class_dict

In [8]:

#, s_test_b
resutls2,resutls3=assessment_model_data_two2(x_test_e,y_test_e, x_test_v,y_test_v,trunk, s_test_v, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

Total number of enrollment samples: 10000
Total number of verification samples: 160051 (160051, 128)
I am 0 - enrollment (10000, 128)
I am 1
[ 70  82  85  86  96 103 106 109 111 115 118 119 125 129 130 131 136 145
 156 159 167 169 173 174 175 176 181 183 184 187 191 192 194 197 199 201
 203 206 207 214 216 217 219 221 225 230 231 233 236 245 249 250 251 257
 260 261 262 265 268 270 277 281 282 284 286 291 296 299 301 303 306 310
 314 322 325 332 336 342 348 354 357 364 365 373 374 384 385 387 402 404
 407 413 414 415 418 419 422 423 433 436]
I am 9
(160051, 128) (10000, 128)
I am 10
(160051, 10000)
70: EER = 0.0290, FMR100 = 0.0861, FMR1000 = 0.2883, FMR10000 = 0.4750, Count = 100
82: EER = 0.0117, FMR100 = 0.0129, FMR1000 = 0.2137, FMR10000 = 0.4338, Count = 100
85: EER = 0.0933, FMR100 = 0.4067, FMR1000 = 0.6089, FMR10000 = 0.7922, Count = 100
86: EER = 0.0156, FMR100 = 0.0733, FMR1000 = 0.6411, FMR10000 = 0.8639, Count = 100
96: EER = 0.0666, FMR100 = 0.4153, FMR1000 = 0.6135, FMR10

(7.7387496426443425,
 7.014847284560699,
 39.97137168644346,
 29.93874354822519,
 66.2031561757628,
 27.001609075804506,
 80.26606862204595,
 22.6130445407728)